# Question 2

# Todo List

<input type="checkbox">Create mazes of dimensions a x b with c loops, where a >= 10 and b >= 10. Save the maze in CSV format. </br>
<input type="checkbox">Place the start and goal locations anywhere inside the maze. </br>
<input type="checkbox">Implement the following search strategies to reach from the start to goal locations: </br>
<input type="checkbox">A* search using Manhattan distance </br>
<input type="checkbox">Breadth-First Search where a cell is allowed to be visited at most thrice. </br>
<input type="checkbox">Compute the amount of time that the agent needs to reach the goal location using both search strategies. </br>
<input type="checkbox">Plot visuals to compare the time that the agent needs to reach the goal using A* and breadth-first search for 5 different maze sizes. </br>
<input type="checkbox">Write a summary of observations and outcomes in a text cell within your .ipynb file. The summary should be a paragraph of no more than 100 words.

In [12]:
%pip install pyamaze

304.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


# Breadth-first Search

In [1]:
from collections import deque

def BFS(m, goal):
    # Initialization
    start = (m.rows, m.cols)
    explored = set([start])
    frontier = deque([start])

    print("original frontier=", frontier)
    print("original explored=", explored)

    bfsPath = {}
    searchPath = []
    visitedCount = {}  # Counter to track visited cells

    # Continue loop as long as frontier is not empty
    while frontier:
        currCell = frontier.popleft()
        searchPath.append(currCell)

        # If current cell == goal, then end
        if currCell == (goal[0], goal[1]):
            break

        # Look into all 4 directions
        for d in "ESNW":
            if m.maze_map[currCell][d]:
                if d == "E":
                    childCell = (currCell[0], currCell[1] + 1)
                elif d == "W":
                    childCell = (currCell[0], currCell[1] - 1)
                elif d == "S":
                    childCell = (currCell[0] + 1, currCell[1])
                elif d == "N":
                    childCell = (currCell[0] - 1, currCell[1])

                if childCell in explored:
                    # Check if the cell has been visited less than 3 times
                    if visitedCount.get(childCell, 0) < 3:
                        visitedCount[childCell] = visitedCount.get(childCell, 0) + 1
                        frontier.append(childCell)
                    continue

                explored.add(childCell)
                frontier.append(childCell)

                print("frontier=", frontier)
                print("explored=", explored)

                bfsPath[childCell] = currCell

    print("=============================================")
    # Print out content of bfsPath
    print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in bfsPath.items()) + "}")

    fwdPath = {}
    cell = (goal[0], goal[1])

    print("=============================================")

    while cell != start:
        fwdPath[bfsPath[cell]] = cell
        cell = bfsPath[cell]

    print("=============================================")
    print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in fwdPath.items()) + "}")

    return fwdPath


# A* Search

In [1]:
from pyamaze import maze,agent,COLOR,textLabel
from queue import PriorityQueue

def h(cell1,cell2):
    x1,y1=cell1
    x2,y2=cell2
    return abs(x1-x2) + abs(y1-y2) 

def aStar(m):
    start=(m.rows,m.cols)
    g_score={cell:float('inf') for cell in m.grid}
    g_score[start]=0
    f_score={cell:float('inf') for cell in m.grid}
    f_score[start]=h(start,(1,1))

    open=PriorityQueue()
    open.put((h(start,(1,1)),h(start,(1,1)),start))
    aPath={}
    
    searchPath = []
    while not open.empty():
        currCell = open.get()[2]
        searchPath.append(currCell)
        
        if currCell == (1,1):
            break
        for d in 'ESNW':
            if m.maze_map[currCell][d] == True:
                if d == 'E':
                    childCell = (currCell[0],currCell[1]+1)
                if d == 'S':
                    childCell = (currCell[0]+1,currCell[1])
                if d == 'N':
                    childCell = (currCell[0]-1,currCell[1])
                if d == 'W':
                    childCell = (currCell[0],currCell[1]-1)
                    
                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score+h(childCell,(1,1))
                
                if temp_f_score < f_score[childCell]:
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_f_score
                    open.put((temp_f_score,h(childCell,(1,1)),childCell))
                    aPath[childCell] = currCell    
    fwdPath={}
    cell=(1,1)
    
    while cell!=start:
        fwdPath[aPath[cell]]=cell
        cell=aPath[cell]
    return fwdPath,aPath,searchPath

# Main Driver Function

## Maze creation

In [2]:
import random
from pyamaze import maze, agent, COLOR

a = 10 #size of maze
b = 10 #size of maze
c = 10 #loop percent

# get random value for goal
goal = random.randint(1,a - 1),random.randint(1,b - 1)
m = maze(a,b) #instantiation of maze
m.CreateMaze(goal[0],goal[1],loopPercent=c,saveMaze=True) #saveMaze exports the maze into a csv file
	# print('maze created')
 
path=BFS(m,goal)
# path=aStar(m) 
# BFS Search
bfsAgent=agent(m,footprints=True)
m.tracePath({bfsAgent:path})

# A* Search
# b=agent(m,footprints=True,color=COLOR.yellow,filled=True)
# m.tracePath({b:path[2]},showMarked=True)
# m.tracePath({a:path[0]})

# l = textLabel(m, "A* Search", (0,0))
m.run()

original frontier= deque([(10, 10)])
original explored= {(10, 10)}
frontier= deque([(9, 10)])
explored= {(9, 10), (10, 10)}
frontier= deque([(9, 10), (10, 9)])
explored= {(9, 10), (10, 10), (10, 9)}
frontier= deque([(10, 9), (10, 10), (8, 10)])
explored= {(9, 10), (10, 10), (8, 10), (10, 9)}
frontier= deque([(10, 9), (10, 10), (8, 10), (9, 9)])
explored= {(9, 10), (9, 9), (8, 10), (10, 10), (10, 9)}
frontier= deque([(10, 10), (9, 10), (10, 9), (9, 10), (9, 10), (8, 9)])
explored= {(9, 10), (9, 9), (8, 10), (10, 10), (8, 9), (10, 9)}
frontier= deque([(10, 9), (10, 10), (8, 10), (9, 9), (8, 10), (9, 9), (8, 10), (9, 9), (7, 9)])
explored= {(9, 10), (9, 9), (8, 10), (7, 9), (10, 10), (8, 9), (10, 9)}
frontier= deque([(10, 9), (10, 10), (8, 10), (9, 9), (8, 10), (9, 9), (8, 10), (9, 9), (7, 9), (8, 8)])
explored= {(9, 10), (8, 8), (9, 9), (8, 10), (7, 9), (10, 10), (8, 9), (10, 9)}
frontier= deque([(8, 8), (10, 9), (8, 9), (8, 9), (8, 9), (7, 10)])
explored= {(9, 10), (8, 8), (9, 9), (7, 1

: 

: 